# HeartAttack Prediction
### 속성 설명
1. Age : 나이
2. Sex : 성별 (1 = 남성, 0 = 여성)
3. cp : 가슴 통증 종류
    - 1: 전형적인 협심증
    - 2: 불규칙적인 협심증
    - 3: 협심증이 아닌 통증
    - 4: 무증상
4. trtbps : 혈압
5. chol : 콜레스트롤 수치
6. fbs : 공복 혈당 (fasting blood sugar > 120 mg/dl) (1 = true; 0 = false)
7. restecg : 공복 심전도 (0 = 정상, 1 = , 2)
8. thalachh : 최대 심박수
9. exang : 운동으로 인한 협심증 (1 = 예; 0 = 아니요)
10. output : 0 = 심장마비 확률 적음, 1 = 심장마비 확률 높음
 
(angina/협심증 : 심장의 혈액 순환로가 되는 관상동맥이 좁아져 갑작스럽게 흉부 통증 또는 압박감을 느끼는 상태)

- 역할 분담 (예정)
    - 이예현 : 데이터 가공, 시각화, 주석 작성
    - 정해성 : 모델 학습, 예측, 평가

- 출처 : https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset

### 데이터 시각화
---


## 머신러닝 모델 학습
---

In [1]:
import pandas as pd

origin_df = pd.read_csv('./heart.csv')
origin_df.drop(['oldpeak', 'slp', 'caa', 'restecg', 'thall'], axis = 1, inplace = True)
origin_df.rename(
    columns = {
        'age' : 'Age',
        'sex' : 'Sex',
        'cp' : 'PainType',
        'trtbps' : 'BloodPressure',
        'chol' : 'CholestoralDensity',
        'fbs' : 'BloodSugar',
        'thalachh' : 'MaxHeartRate',
        'exng' : 'IsExerciseInduced',
        'output' : 'Result',
    },
    inplace = True
)

df_feature = origin_df.drop(['Result'], axis=1, inplace=False)
df_label = origin_df['Result']

### One-Hot Encoding

In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

atomic_col_names = ['Sex', 'PainType', 'BloodSugar', 'IsExerciseInduced']

df_feature_oh = pd.get_dummies(df_feature, columns=atomic_col_names)
df_feature_oh.head()

,Age,BloodPressure,CholestoralDensity,MaxHeartRate,Sex_0,Sex_1,PainType_0,PainType_1,PainType_2,PainType_3,BloodSugar_0,BloodSugar_1,IsExerciseInduced_0,IsExerciseInduced_1
0,63,145,233,150,0,1,0,0,0,1,0,1,1,0
1,37,130,250,187,0,1,0,0,1,0,1,0,1,0
2,41,130,204,172,1,0,0,1,0,0,1,0,1,0
3,56,120,236,178,0,1,0,1,0,0,1,0,1,0
4,57,120,354,163,1,0,1,0,0,0,1,0,0,1


#### 불필요한 요소 포함하고 모델 학습 및 포함 안 하고 모델 학습

In [7]:
from sklearn.model_selection import train_test_split

# 학습용 데이터와 테스트용 데이터로 분리 (8 : 2)
def split(feature, label):
    return train_test_split(feature, label, test_size=0.2, random_state=11)

In [8]:
# 로지스틱 회귀 사용
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import  accuracy_score

# 분할하고 표준화를 하면 행 순서가 맞지 않아서 join을 했을 때 NaN이 뜨는 현상이 있음
# 그래서 학습 데이터 전체를 표준화 한 후 분할을 할 예정

con_indices = ['Age', 'BloodPressure', 'CholestoralDensity', 'MaxHeartRate'] # 연속적인 데이터프레임 인덱스

x_scaled = StandardScaler().fit_transform(df_feature[con_indices]) # 연속적인 열만 표준화

df_scaled_feature = pd.DataFrame(x_scaled, columns=con_indices) # DataFrame으로 변환

df_scaled_feature = df_scaled_feature.join(df_feature_oh.drop(con_indices, axis=1, inplace=False)) # 표준화 된 데이터 프레임과 기존 것과 합침

x_train_s, x_test_s, y_train_s, y_test_s = split(df_scaled_feature, df_label) # 표준화 된 데이터 프레임과 레이블을 학습/테스트 데이터로 분리

## 로지스틱 회귀를 이용해서 모델 학습
model = LogisticRegression()

model.fit(x_train_s, y_train_s) # 모델 학습

prediction = model.predict(x_test_s) # 예측

pred_score = accuracy_score(y_test_s, prediction) # 평가

print('정확도 :', pred_score)

정확도 : 0.819672131147541


In [9]:
# DecisionTree 사용
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=11)

dt_clf.fit(x_train_s, y_train_s)

pred = dt_clf.predict(x_test_s)

print('정확도 :', accuracy_score(y_test_s, pred))

정확도 : 0.5737704918032787


In [6]:
# KFold 교차 검증
from sklearn.model_selection import KFold, cross_val_score

cv = KFold(n_splits=4)
